In [44]:
import numpy as np
import time
from IPython.display import clear_output
# direction is 0 = horizontal, 1 = vertical

gamma, alpha, beta, theta = 0.5, 0.5, 0.5, 0.5

class Particle:
    
    num_particles = 0
    
    def __init__(self, road, direction):
        
        self.id = Particle.num_particles
        Particle.num_particles += 1
        
        self.gamma = gamma
        
        self.position = [0, road.position] if not road.direction else [road.position, 0]
        self.direction = road.direction
        self.active = True
    
    def update_position(self):
        
        if not self.direction:
            self.position[0] += 1
        else:
            self.position[1] += 1
        
    def turn(self):
        
        self.direction = int(not self.direction)
        self.update_position()
    
    
class Road:
    
    def __init__(self, direction, position, dimension):
        
        self.alpha = alpha
        self.beta = beta
        self.dimension = dimension
        
        self.grid = np.zeros((self.dimension,))
        self.particles = []
        self.direction = direction
        self.position = position
    
    def move_particle(self, particle):
    
        if not self.direction:
            curr_pos = particle.position[0]
        else:
            curr_pos = particle.position[1]
        if not curr_pos == self.dimension - 1 and not self.grid[curr_pos + 1]:
            particle.update_position()
            self.update()
            return '{} moved to {} at time '.format(particle.id, particle.position)
        return None
    
    def update(self):
        
        self.grid = np.zeros((self.dimension,))
        if not self.direction:
            for p in self.particles:
                self.grid[p.position[0]] = 1
                
        else:
            for p in self.particles:
                self.grid[p.position[1]] = 1
                
    def add_particle(self):
        
        if not self.grid[0]:
            new_particle = Particle(self, self.direction)
            self.particles.append(new_particle)
            self.update()
            return '{} added at {} at time '.format(new_particle.id, new_particle.position)
        return None
        
    def remove_particle(self):
        
        if self.grid[-1]:
            curr_particle = self.particles.pop(-1)
            self.update()
            return '{} exited from {} at time '.format(curr_particle.id, curr_particle.position)
        return None
        
    
class Intersection:
    
    def __init__(self, position):
        
        self.theta = theta
        
        self.position = position
        self.light = 0
        
    def change_light(self):
        
        self.light = int(not self.light)
    
    
class Environment:
    
    def __init__(self, num_horz_roads, num_vert_roads, dimension):
        
        self.dimension = dimension
        
        horizontal_positions = np.random.choice(range(dimension), num_horz_roads, replace=False)
        vertical_positions = np.random.choice(range(dimension), num_vert_roads, replace=False)
        
        self.horz_roads = [Road(0, p, dimension) for p in horizontal_positions]
        self.vert_roads = [Road(1, p, dimension) for p in vertical_positions]
        
        grid = np.zeros((dimension, dimension))
        for road in self.horz_roads:
            grid[road.position, :] = road.grid
        for road in self.vert_roads:
            grid[:, road.position] = road.grid
            
        self.grid = grid
        
        self.intersections = []
        for y in self.horz_roads:
            for x in self.vert_roads:
                new_intersection = Intersection((x.position, y.position))
                self.intersections.append(new_intersection)
                
    def get_param(self):
        
        horz_param = sum([sum([p.gamma for p in road.particles]) for road in self.horz_roads])
        vert_param = sum([sum([p.gamma for p in road.particles]) for road in self.vert_roads])
        move_param = horz_param + vert_param
        add_param = sum([road.alpha for road in self.horz_roads + self.vert_roads])
        remove_param = sum([road.beta for road in self.horz_roads + self.vert_roads])
        light_param = sum([intersection.theta for intersection in self.intersections])
        
        return move_param + add_param + remove_param + light_param
    
    def choose_action(self):
        
        actions = ['move', 'add', 'remove', 'light']
        
        horz_param = sum([sum([p.gamma for p in road.particles]) for road in self.horz_roads])
        vert_param = sum([sum([p.gamma for p in road.particles]) for road in self.vert_roads])
        move_param = horz_param + vert_param
        add_param = sum([road.alpha for road in self.horz_roads + self.vert_roads])
        remove_param = sum([road.beta for road in self.horz_roads + self.vert_roads])
        light_param = sum([intersection.theta for intersection in self.intersections])
        
        probs = [move_param, add_param, remove_param, light_param]
        probs = probs / np.sum(probs)
        
        return np.random.choice(actions, p=probs)
        
    def move_particle(self):
        
        # choose road
        horz_p = [sum([p.gamma for p in road.particles]) for road in self.horz_roads]
        vert_p = [sum([p.gamma for p in road.particles]) for road in self.vert_roads]
        probs = np.array(horz_p + vert_p)
        probs = probs / np.sum(probs)
        road = np.random.choice(self.horz_roads + self.vert_roads, p=probs)
        
        # choose particle
        probs = np.array([p.gamma for p in road.particles])
        probs = probs / np.sum(probs)
        particle = np.random.choice(road.particles, p=probs)
        
        # update

        result = road.move_particle(particle)
        self.update_grid(road)
        return result
        
    def add_particle(self):
        
        roads = self.horz_roads + self.vert_roads
        probs = np.array([road.alpha for road in roads])
        probs = probs / np.sum(probs)
        road = np.random.choice(roads, p=probs)
        result = road.add_particle()
        self.update_grid(road)
        return result
        
    def remove_particle(self):
        
        roads = self.horz_roads + self.vert_roads
        probs = np.array([road.beta for road in roads])
        probs = probs / np.sum(probs)
        road = np.random.choice(roads, p=probs)
        result = road.remove_particle()
        self.update_grid(road)
        return result
        
    def update_light(self):
        
        probs = np.array([i.theta for i in self.intersections])
        probs = probs / np.sum(probs)
        intersection = np.random.choice(self.intersections, p=probs)
        intersection.change_light()
        
        return 'light switched at {} at '.format(intersection.position)
        
    def update_grid(self, road):
        
        if not road.direction:
            self.grid[road.position, :] = road.grid
        else:
            self.grid[:, road.position] = road.grid

    
class Simulation:
    
    def __init__(self):
        
        self.env = Environment(2, 2, 10)
        self.logger = Logger()
        self.clock = 0
        self.log = []
        
    def visualize(self):
        
        clear_output(wait=True)
        print(self.env.grid)
    
    def take_action(self):
        
        param = self.env.get_param()
        time_lapse = np.random.exponential(scale=1/param)
        time.sleep(time_lapse)
        self.clock += time_lapse
        
        action = self.env.choose_action()
        
        if action == 'move':
            result = self.env.move_particle()
        elif action == 'add':
            result = self.env.add_particle()
        elif action == 'remove':
            result = self.env.remove_particle()
        elif action == 'light':
            result = self.env.update_light()
            
        return result
        
    def run(self):
        
        while self.clock < 10:
            result = self.take_action()
            if result:
                self.log.append(result + str(self.clock))
            self.visualize()
        [print(line) for line in self.log]

In [45]:
sim = Simulation()
sim.run()

[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
light switched at (0, 8) at 0.17383166525609983
light switched at (0, 5) at 0.7772038337512155
light switched at (0, 5) at 0.9486571442886863
0 added at [0, 8] at time 1.1733552029224812
1 added at [0, 0] at time 1.2188765990867914
1 moved to [0, 1] at time 1.3297710036930994
0 moved to [1, 8] at time 1.7388918746625157
2 added at [6, 0] at time 2.1813918203824345
2 moved to [6, 1] at time 2.2093997867989406
3 added at [0, 0] at time 2.2506196019696003
1 moved to [0, 2] at time 2.296835479744367
light switched at (6, 8) at 2.324712398137234
light switched at (0, 5) at 2.617956856162949
1 moved to [0, 3] at time 3.0552215942094145
light switched at (0, 5) at 3.6